In [9]:
import pandas as pd

# Define the path to the input and output Excel files
input_file = r"D:\Grenoble_university\6_month_internship\working space\data\combined_1_year_data_with_flexibility.xlsx"
output_file = r"D:\Grenoble_university\6_month_internship\working space\6) Flexibility potentials\annual_energy_consumption_ex1.xlsx"

# List of sheets to process
sheets = ['2000900', '2000901', '2000902', '2000903']

# Process each sheet
with pd.ExcelWriter(output_file) as writer:
    for sheet in sheets:
        # Read the data from the current sheet
        df = pd.read_excel(input_file, sheet_name=sheet)
        
        # Convert the 'date' column to datetime format
        df['date'] = pd.to_datetime(df['date'])
        
        # Calculate the interval duration in hours
        df = df.sort_values(by='date')  # Ensure the data is sorted by date
        interval_seconds = (df['date'].iloc[1] - df['date'].iloc[0]).total_seconds()  # Interval in seconds
        interval_hours = interval_seconds / 3600  # Convert seconds to hours
        
        # Identify the device columns (assuming all columns except 'Weekday' and 'date' are devices)
        device_columns = [col for col in df.columns if col not in ['Weekday', 'date']]
        
        # Convert device columns to numeric, coerce errors to NaN
        df[device_columns] = df[device_columns].apply(pd.to_numeric, errors='coerce')
        
        # Calculate average power consumption for each appliance
        average_power_dict = {}
        for device in device_columns:
            average_power = df[device].sum() / len(df)  # Calculate average power
            average_power_dict[device] = average_power
        
        # Convert average power consumption to energy consumption for each appliance
        energy_consumption_dict = {}
        for device, average_power in average_power_dict.items():
            energy_consumption = average_power * interval_hours  # Energy in kWh
            energy_consumption_dict[device] = energy_consumption
        
        # Convert the energy consumption to DataFrame for the current sheet
        energy_consumption_df = pd.DataFrame.from_dict(energy_consumption_dict, orient='index', columns=['Energy_kWh']).T
        
        # Save the results to a new sheet in the Excel file
        energy_consumption_df.to_excel(writer, sheet_name=sheet)

print(f"Annual energy consumption data saved to {output_file}")


Annual energy consumption data saved to D:\Grenoble_university\6_month_internship\working space\6) Flexibility potentials\annual_energy_consumption_ex2.xlsx


In [1]:
import pandas as pd

# Function to convert power values to energy values
def power_to_energy(df):
    # Assuming the power values are in watts and the time step is 10 minutes
    # Energy (in kWh) = Power (in W) * Time (in hours)
    # Time = 10 minutes = 10/60 hours = 1/6 hours
    time_step_hours = 1/6
    
    # Copy the dataframe to avoid modifying the original
    df_energy = df.copy()
    
    # Convert all columns except 'Weekday' and 'date' to numeric, coerce errors to NaN and fill NaN with 0
    df_energy.iloc[2:, 2:] = df_energy.iloc[2:, 2:].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # Multiply all power values by time_step_hours to convert to energy (kWh)
    df_energy.iloc[2:, 2:] = df_energy.iloc[2:, 2:] * time_step_hours
    
    return df_energy

# File path to the input Excel file
input_file_path = r'D:\Grenoble_university\6_month_internship\working space\data\combined_1_year_data_with_flexibility.xlsx'
# File path to the output Excel file
output_file_path = r'D:\Grenoble_university\6_month_internship\working space\6) Flexibility potentials\energy_values.xlsx'

# Read the Excel file
xls = pd.ExcelFile(input_file_path)

# Dictionary to hold the dataframes for each sheet
energy_sheets = {}

# Loop through each sheet and convert power to energy
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    df_energy = power_to_energy(df)
    energy_sheets[sheet_name] = df_energy

# Write the energy values to a new Excel file
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, df_energy in energy_sheets.items():
        df_energy.to_excel(writer, sheet_name=sheet_name, index=False)

print("Energy values have been successfully saved to", output_file_path)


Energy values have been successfully saved to D:\Grenoble_university\6_month_internship\working space\6) Flexibility potentials\energy_values.xlsx
